In [ ]:
#######################################################
# Script:
#    trainRandomForest.py
# Usage:
#    python trainRandomForest.py
# Description:
#    Build the prediction model based on training data
# Authors:
#    Jackie Chu,   cchu@salesforce.com
#    Jasmin Nakic, jnakic@salesforce.com
#######################################################

import numpy as np
from sklearn import tree
from sklearn import ensemble
from sklearn.externals import joblib

In [ ]:
# Enable or disable debug printing
debugFlag = False

# Feature list
perfCols = ["PageTime_ms","TotalServerTime_ms","TotalBrowserTime_ms","Action_count","Api_count","Db_count","DbTime_ms","Xhr_count"]

# Model options
maxDepth = 0
numEstimators = 15

In [ ]:
# Add a set of columns to the destination
def addColumns(dest, src, colNames):
    # Initialize temporary array
    tmpArr = np.empty(src.shape[0])
    cols = 0
    # Copy column content
    for name in colNames:
        if cols == 0: # first column
            tmpArr = np.copy(src[name])
            tmpArr = np.reshape(tmpArr,(-1,1))
        else:
            tmpCol = np.copy(src[name])
            tmpCol = np.reshape(tmpCol,(-1,1))
            tmpArr = np.append(tmpArr,tmpCol,1)
        cols = cols + 1
    return np.append(dest,tmpArr,1)
#end addColumns

In [ ]:
# Generate Random Forest model
def genModel(data,colList,modelName):
    # Prepare the data for the model
    X = np.zeros(data.shape[0])
    X = np.reshape(X,(-1,1))
    X = addColumns(X,data,colList)
    if debugFlag:
        print("X 0: ", X[0:5])
    Y = np.copy(data["Status"])
    if debugFlag:
        print("Y 0: ", Y[0:5])

    # Build the model based on training data
    model = ensemble.RandomForestClassifier(oob_score=True)
    if numEstimators > 0 :
        model.set_params(n_estimators=numEstimators)
    if maxDepth > 0 :
        model.set_params(max_depth=maxDepth)
    print(model.fit(X, Y))

    print("MODEL: ", model)
    print("N_FEATURES: ", model.n_features_)
    print("MAX_DEPTH: ", model.max_depth)
    print("NAMES: ", data.dtype.names)
    print("FEATURE_IMPORTANCES: ", model.feature_importances_)
    print("N_ESTIMATORS: ", model.n_estimators)
    print("OOB_DECISION_FUNCTION: ", model.oob_decision_function_)
    print("OOB_SCORE: ", model.oob_score_)

    P = model.predict(X)
    print("SCORE values: ", model.score(X,Y))
    if debugFlag:
        print("P 0-5: ", P[0:5])

    # Write the model to the file
    modelFileName = modelName+".model"
    joblib.dump(model,modelFileName)
    return P
#end genModel

In [ ]:
-

In [ ]:
# Start
inputFileName = "PerfRun_TrainingData.csv"
outputFileName = "PerfRun_TrainingRandomForest.txt"
modelName = "PerfRandomForest"

# All input columns - data types are strings, float and int
trainData = np.genfromtxt(
    inputFileName,
    delimiter=',',
    names=True,
    dtype=("|U20",int,int,int,int,int,int,int,int)
)
if debugFlag:
    print("trainData 0: ", trainData[0:5])

P = genModel(trainData,perfCols,modelName)
writeResult(outputFileName,trainData,P)